### 采用Sarsa算法求解该问题中的最优路径

In [90]:
import gym
import matplotlib
import numpy as np
from collections import defaultdict
from gym.envs.toy_text.cliffwalking import CliffWalkingEnv

In [91]:
'''
新版本gym在创建环境时需要指定render_mode这个参数
（1）"human"：渲染到当前显示设备或终端，适合人类观察。
（2）"ansi"：文本渲染模式，适合简化查看环境变化的情况，尤其是对状态变化进行跟踪。
（3）"rgb_array"：图像渲染模式，适合需要视觉反馈的场景，适用于图形化环境。
（4）None：无渲染模式，适合对渲染无需求的情况，主要关注计算效率
'''
env = CliffWalkingEnv(render_mode="ansi")   

In [92]:
# epsilon_greedy返回policy函数
# policy函数的输入是状态，输出是根据epsilon_greedy采取各个行动的概率
def epsilon_greedy(Q, epsilon, nA):
    def policy(state):
        # state可能会包含字典(如：(36, {'prob': 1}))，此处只选取state中有价值的信息，即元组的第一个元素
        if isinstance(state, tuple):
            state = state[0]  
        else:
            state = state 
        A_prob = np.ones(nA) * epsilon / nA
        best_action = np.argmax(Q[state])
        A_prob[best_action] += (1 - epsilon)
        return A_prob
    return policy 

In [93]:
def sarsa(env, n, discount=1.0, epsilon=0.1, alpha=0.5):
    # Q = {state:[action1-value, action2-value]}
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    policy = epsilon_greedy(Q, epsilon, env.action_space.n)
    for i in range(n):
        state = env.reset()
        while(True):
            # 在状态state下，根据策略policy，计算行动概率
            prob = policy(state)
            # 采取行动：action
            action = np.random.choice(np.arange(len(prob)), p=prob)
            # 行动action导致下一个状态next_state, env.step(action)的格式为：(24, -1, False, False, {'prob': 1.0})
            next_state, reward, done, _, info = env.step(action)
            # 在状态next_state下，根据策略policy，计算行动概率
            next_action_prob = policy(next_state)
            # 采取行动：next_action
            next_action = np.random.choice(np.arange(len(next_action_prob)), p=next_action_prob)

            # state可能会包含字典(如：(36, {'prob': 1}))，此处只选取state中有价值的信息，即元组的第一个元素
            if isinstance(state, tuple):
                state = state[0]  
            else:
                state = state
            td_error = reward + discount * Q[next_state][next_action] - Q[state][action]
            Q[state][action] = Q[state][action] + alpha * td_error
            
            if done:
                break  
            
            state = next_state
            action = next_action 
    return Q

In [94]:
def td_render(Q):
    state = env.reset()
    # state可能会包含字典(如：(36, {'prob': 1}))，此处只选取state中有价值的信息，即元组的第一个元素
    if isinstance(state, tuple):
        state = state[0]  
    else:
        state = state
    while True:
        next_state, reward, done, _, info = env.step(np.argmax(Q[state]))
        env.render()
        if done:
            break
        state = next_state

In [95]:
Q = sarsa(env, 1000)
td_render(Q)